# Bijgebouw Comparison

In [4]:
import pandas as pd
import numpy as np
import folium

In [5]:
df1 = pd.read_excel(r"C:\Users\Gebruiker\Desktop\GITHUB\Geomatics\book21_edited.xlsx")

In [6]:
df1.head()

,Nummer,H,Address,Latitude,Longitude
0,210004,het plaatsen van een dakkapel,"1, Zeistpad, Stedenwijk, Almere Stad, Almere, ...",52.363841,5.203593
1,210039,het plaatsen van een dakkapel,"34, Barneveldpad, Stedenwijk, Almere Stad, Alm...",52.362509,5.208172
2,210092,het plaatsen van een dakkapel,"35, Breitnerstraat, Tussen de Vaarten, Almere,...",52.379402,5.254923
3,210172,het plaatsen van een dakkapel,"12, Beuvenstraat, Waterwijk, Almere, Flevoland...",52.386438,5.221712
4,210227,Het plaatsen van 2 dakkapellen,"8, Kaatsheuvelstraat, Stedenwijk, Almere, Flev...",52.357661,5.201078


In [8]:
locations = df1[['Latitude','Longitude']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist[7]

[52.3885517, 5.2397798]

In [20]:
loc = 'Bijgebouw Aanvragen 2021'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   


from branca.element import Figure
fig=Figure(width=1000,height=1000)

map = folium.Map(location=[52.371320, 5.222381], zoom_start=12)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df1['H'][point]).add_to(map)


fig2=Figure(width=1000,height=800)

fig2.add_child(map)
folium.TileLayer('Stamen Terrain').add_to(map)
folium.TileLayer('Stamen Toner').add_to(map)
folium.TileLayer('Stamen Water Color').add_to(map)
folium.TileLayer('cartodbpositron').add_to(map)
folium.TileLayer('cartodbdark_matter').add_to(map)
folium.LayerControl().add_to(map)

In [21]:
map.get_root().html.add_child(folium.Element(title_html))

map.save('map-with-title.html')
map

In [11]:
map2 = folium.Map(location=[38.9, -77.05], tiles='CartoDB dark_matter', zoom_start=11)

marker_cluster = folium.MarkerCluster().add_to(map2)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df_counters['Name'][point]).add_to(marker_cluster)
map2

The result map point are a bit easier to see, but I'm not sure I'm a bit fan of the dark color of the CartoDB dark_matter tiles. I think I prefer a lighter color for this map. For the next map, I decided to try out the Stamen Terrain tiles. I also decided to change the look of the point icons. I changed their color to dark blue. I also tried to add a bicycle icon. The bicycle icon was not available in version 0.2.1 of Folium, so I instead went with a pedestrian looking icon.

In [12]:
map2 = folium.Map(location=[38.9, -77.05], tiles='Stamen Terrain', zoom_start=11)

marker_cluster = folium.MarkerCluster().add_to(map2)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df_counters['Name'][point], icon=folium.Icon(color='darkblue', icon_color='white', icon='male', angle=0, prefix='fa')).add_to(marker_cluster)
map2

The terrain map is an improvement over the dark map, but I still found the green coloring for natural areas to be distracting from the main objective of the map. I now also want to differentiate the counters by region. To do that, I decided to create a function to assign a unique color to each region.

In [13]:
def regioncolors(counter):
    if counter['region'] == 'Arlington':
        return 'green'
    elif counter['region'] == 'Alexandria':
        return 'blue'
    elif counter['region'] == 'DC':
        return 'red'
    else:
        return 'darkblue'
df_counters["color"] = df_counters.apply(regioncolors, axis=1)
df_counters.head()

,ID,Name,latitude,longitude,region,color
0,33,110 Trail,38.885315,-77.065022,Arlington,green
1,30,14th Street Bridge,38.874260,-77.044610,Arlington,green
2,43,15th Street NW,38.907470,-77.034610,DC,red
3,32,Arlington Mill Trail,38.845610,-77.096046,Arlington,green
4,24,Ballston Connector,38.882950,-77.121235,Arlington,green


I then changed the point icon color for each counter based on its region. I also switched the basemap to the lighter CartoDB positron tiles. While I was at it, I also realized it would be a really good idea to include the counter ID number on the map since you need that number when making specific API calls for counter data.

In [14]:
map3 = folium.Map(location=[38.9, -77.05], tiles='CartoDB positron', zoom_start=11)

marker_cluster = folium.MarkerCluster().add_to(map3)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup='ID:'+df_counters['ID'][point]+' '+df_counters['Name'][point], icon=folium.Icon(color=df_counters["color"][point], icon_color='white', icon='male', angle=0, prefix='fa')).add_to(marker_cluster)
map3

I'm fairly please with this last map. In the future, I would like to change the symbols on the pointers based on whether the particular sensor can distiguish between bicycles and pedestrians. It is also possible to attach bar charts or other graph types to these pointers. In addition to point maps, Folium can also create choropleth maps for displaying regional statistics using color gradients.